# ABOUT

__Author__: Pat McCornack

__Date__: 4/9/24

__Purpose__: Train model to update FVT using most recent FDist data. 

------

In [62]:
import os
import datetime as dt
from joblib import dump, load
from itertools import combinations

import numpy as np
import pandas as pd
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import TargetEncoder, StandardScaler
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier


import matplotlib.pyplot as plt




## Define Paths
Define set of filepaths to conveniently switch between working off local files or the PNNL drive. Set active_data_dir to either local_data_dir or pnnl_data_dir depending on which you're working off of. 

In [50]:
local_root_dir = r"C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\fuelscape_modeling"
pnnl_root_dir = r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\fuelscape_modeling"

# Define which data directory to work off of
active_root_dir = local_root_dir

paths_dict = {
    "out_base_dir" : os.path.join(active_root_dir, r"model_outputs\tabular"),  # Where to save result outputs 
    "ref_data_dir" : os.path.join(active_root_dir, r"..\LF_raster_data\_tables"),  # Location of LF csvs (e.g. LF22_FVT_230.csv)
    "sample_points_dir" : os.path.join(active_root_dir, r"sample_points"),  # Location of shapefile to train model on
    "model_dir" : os.path.join(active_root_dir, "models")
}

sample_points_fnames = {
    "LF22_F40" : "LF22_F40_sample_points_2024-05-29_100k.shp",
    "LF22_FVT" : "LF22_FVT_Disturbed_sample_points_2024-05-29_200k.shp",
    "LF22_FVC" : "LF22_FVT_Disturbed_sample_points_2024-05-29_200k.shp",
    "LF22_FVH" : "LF22_FVT_Disturbed_sample_points_2024-05-29_200k.shp"
}



# __Functions__
----


## __Create a Directory to Output Modeling Results__
Names the output directory using the datetime that the script was run. 
Returns the name of the directory. The returned directory is used to output the trained model and/or results. 

In [51]:
def make_dir(base_dir, new_dir_name='model_results'):
        """
        Returns path to a directory created at the specified base_dir location. 

        The name of the created directory can optionally be specified using the dir_name argument. 
        """

        datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        output_dir = os.path.join(base_dir, new_dir_name + "_" + datetime)

        os.makedirs(output_dir)
        return output_dir

## __Pre-Process the Data__

### __Create Data Dictionaries to Append Features__
Some features are separate attributes of the LANDFIRE dataset (e.g. BPS Fire Regime) and others are useful for results analysis (e.g. FDst attributes). These can be mapped to points using LANDFIRE CSVs. The below creates dictionaries to perform that mapping. 

This function is called by join_features.

In [52]:
def model_read_ref_data(ref_data_dir=paths_dict["ref_data_dir"]):
    """
    Returns a dictionary of dictionaries of mappings between LANDFIRE raster values and other attributes associated with those values. 
    """
    data_dir = ref_data_dir
    BPS_fname = "LF20_BPS_220.csv"
   
    # Create empty dictionary
    LF_ref_dicts = {}

    # Get BPS reference dictionary
    BPS_df = pd.read_csv(os.path.join(data_dir, BPS_fname))
    LF_ref_dicts["BPS_NAME"] = dict(BPS_df[['VALUE', 'BPS_NAME']].values)
    LF_ref_dicts["BPS_FRG_NE"] = dict(BPS_df[['VALUE', 'FRG_NEW']].values)

    return LF_ref_dicts
                         
        

### __Append Features using Data Dictionaries__
Append in selected features using the LANDFIRE data dictionaries.

__Note:__ Items in feature_list must be in the source_layers dictionary. 

In [53]:
def model_join_features(sample_points, feature_list = ['BPS_NAME']):
    """
    Returns the sample_points layer with the features in feature_list appended. 

    Items in feature_list must be in the source_layers dictionary.  
    """
    
    LF_ref_dicts = read_ref_data()
    
    source_layers = {
        'BPS_NAME' : 'BPS',
        'BPS_FRG_NE' : 'BPS',
        'FDST_TYPE' : 'LF22_FDST',
        'FDST_SEV' : 'LF22_FDST', 
        'FDST_TSD' : 'LF22_FDST'
    }

    # Iterate through feature_list and append features to sample_points
    for feature in feature_list:
        sample_points[feature] = sample_points[source_layers[feature]].map(LF_ref_dicts[feature]).copy()

    return sample_points

### __Low Count Filter__
Some classes have exceedingly low representation in the dataset. In order to run train_test_split, the target class must have a count greater than 1. The following will filter out classes with counts of 1. Given the size of the dataset excluding these classes will not detrimentally impact model performance.

In [54]:
def low_count_filter(sample_points, target):  
    """
    Returns modified sample_points dataframe. Classes in the target feature that have counts of 1 are removed. 
    """  
    # Group by FVT
    obs_counts = sample_points.groupby(target).count()

    # Identify FVTs with low observation counts
    low_count_classes = obs_counts[obs_counts.iloc[:,0] < 5].index.tolist()

    # Remove those FVTs from sample_points
    sample_points = sample_points.loc[~sample_points[target].isin(low_count_classes)]

    return sample_points

### __Prepare the Sample Points__
Before training the model, the sample points need to be filtered. Filtering steps are:
- Remove near points - defined as points that are less than 70m apart.
- Remove Null points (-9999/-1111 values) - these will not be updated in the final raster. 
- Remove undisturbed values - the model is intended to update points that have been disturbed in the last 10 years. 
- Remove points with agricultural or developed FVT classes - these are updated using datasets not used for other classes. 
- Remove observations with classes that have very low representation (less than 5 observations in the class) in the dataset. This is done to allow train_test_split to run. 
- Optionally, join in additional features. feature_list may be modified to select which features to add in (see join_features for further details). 

In [55]:
def model_data_prep(sample_point_fpath, target):
    """
    Returns dataframe of processed sample points. Sample points are read in from shapefile. Processing includes filtering out null points and developed/agricultural FVTs as 
    well as appending additional features. 
    """

    # Read in gdf
    sample_points = gpd.read_file(sample_point_fpath)

    # Filter out near points
    sample_points = sample_points.loc[sample_points['NEAR_FID'] == -1]

    # Drop unneeeded columns if present
    sample_points = sample_points.drop(['Classified', 'GrndTruth', 'NEAR_FID', 'NEAR_DIST'], axis=1,
                                       errors='ignore')
    
    # Remove observations with -9999/-1111 in any field 
    sample_points = sample_points.loc[~sample_points.isin([-1111, -9999]).any(axis=1)]

    # Join in additional features
    sample_points = join_features(sample_points)

    # Drop non-disturbed values if not predicting F40
    if target != 'LF22_F40':     
        # Filter out points that weren't disturbed
        sample_points = sample_points.loc[sample_points['LF22_FDST'] != 0]

    # Filter classes based on target
    if target == 'LF22_F40':
        F40_NB = [91, 93]
        sample_points = sample_points.loc[~sample_points['LF22_F40'].isin(F40_NB)]

    elif target == 'LF22_FVT':
        developed_fvt = list(range(20,33)) + list(range(2901,2906))
        ag_fvt = [80, 81, 82] + list(range(2960, 2971))
        fvt_filter = developed_fvt + ag_fvt
        sample_points = sample_points.loc[~sample_points['LF22_FVT'].isin(fvt_filter)]

    elif target == 'LF22_FVH' or target == 'LF22_FVC':
        filter = list(range(20, 70)) + list(range(80, 86))
        sample_points = sample_points.loc[~sample_points[target].isin(filter)]

    # Filter out classes in the target feature with very low counts
    sample_points = low_count_filter(sample_points, target)

    return sample_points


## __Instantiate Estimators__

### __Histogram Based Gradient Boosting Classifier__
Scikit-learn implementation of Histogram Based Gradient Boosting Classifier

In [56]:
def histGradientBoostingClassifier(categorical_feature_list, class_weight='balanced', seed=1234):
    """
    Returns specified histogram-based gradient boosting classifier. 
    """

    hgb_classifier = HistGradientBoostingClassifier(
        categorical_features=categorical_feature_list,  # Natively handle categorical variables
        class_weight=class_weight,
        random_state=seed,
        learning_rate=0.01,
        max_iter=100
    )

    return hgb_classifier

## __Train Model__
Trains and returns HGBC model provided data. Subsets the training data to a provided list of predictors and predicts the specified target feature.

__Note__: Aspect, Elevation, and Slope are the only continuous LANDFIRE datasets, therefore any feature not in that list is assumed to be categorical.

In [57]:
def fit_model(train_data, predictors, target):
    """
    Returns trained model of specified type given a set of predictors. 
    """

    class_weight = "balanced"

    # Get list of predictors for run
    cat_features = [x for x in predictors if x not in ['ASPECT', 'ELEVATION', 'SLOPE']]  # Categorical features for HGBC models  
    
    # Separate training data predictors/response
    y_train = train_data[target].copy()
    X_train = train_data[predictors].copy()

    # Instantiate the model
    model = histGradientBoostingClassifier(categorical_feature_list=cat_features, class_weight=class_weight)
      
    # Fit the model with the training data
    print(f'Fitting {target} Model...')
    model.fit(X_train, y_train)

    # Return the fit model
    return model         
    

## __Evaluate Model__
Provided a trained model, test data, a set of predictors, and a target - return a list of predictions metrics on prediction accuracy.

In [58]:
def eval_model(model, test_data, predictors, target):
    """
    Returns a dictionary containing 1. 'metrics' : a list of metrics quantifying model performance, and 2. 'predictions' : a list of predictions corresponding to 
    each observation in the sample_points data.
    """
       
    # Separate the predictors from target
    y_test = test_data[target].copy()
    x_test = test_data[predictors].copy()

    # Perform prediction
    y_pred = model.predict(x_test)

    # Get metrics
    accuracy = round(accuracy_score(y_test, y_pred), 3)
    bal_acc = round(balanced_accuracy_score(y_test, y_pred),3)
    recall = round(recall_score(y_test, y_pred, average='macro'), 3)
    precision = round(precision_score(y_test, y_pred, average='macro'), 3)
    f1 = round(f1_score(y_test, y_pred, average='macro'), 3)

    print(f"Predictors: {predictors}")
    print(f"Accuracy: {accuracy}")
    print(f"Balanced Accuracy: {bal_acc}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F1: {f1}")

    return {
        "metrics" : [accuracy, bal_acc, recall, precision, f1, predictors],
        "predictions" : y_pred
    }


In [59]:
def train_model_wrapper(target, sample_points_fname):
    predictors_dict = {
        'LF22_FVT' : ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH'],
        'LF22_FVC' : ['LF22_FVT', 'LF20_FVC', 'LF22_FDST', 'ZONE', 'BPS_NAME'],
        'LF22_FVH' : ['LF22_FVT', 'LF20_FVH', 'LF22_FDST', 'ZONE', 'BPS_NAME'],
        'LF22_F40' : ['LF22_FVT', 'LF22_FVH', 'LF22_FVC', 'LF22_FDST', 'ZONE', 'BPS_FRG_NE']
    }

    # Define path to sample points
    sample_points_fpath =  os.path.join(paths_dict['sample_points_dir'], sample_points_fname)

    # Read in and prepare data
    sample_points = data_prep(sample_points_fpath, target)

    # Train model
    model = fit_model(train_data=sample_points, predictors=predictors_dict[target], target=target)

    # Save model 
    datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    model_fname = f"{target}_HGBC_model_{datetime}"
    model_fpath = os.path.join(paths_dict['model_dir'], model_fname)
    dump(model, model_fpath)
    print(f"{target} model written to: {model_fpath}")

# __Main__

----

## Train Model
Specify the models to train by adjusting the 'targets' list, then run the cell.

In [60]:
# Specify the models to train
targets = ["LF22_FVC", "LF22_FVH", "LF22_F40"]

# Train the models
for target in targets:
    train_model_wrapper(target, sample_points_fnames[target])

Fitting LF22_FVC Model...
LF22_FVC model written to: C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\fuelscape_modeling\models\LF22_FVC_HGBC_model_2024-05-29_12-06-42
Fitting LF22_FVH Model...
LF22_FVH model written to: C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\fuelscape_modeling\models\LF22_FVH_HGBC_model_2024-05-29_12-07-58
Fitting LF22_F40 Model...
LF22_F40 model written to: C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\fuelscape_modeling\models\LF22_F40_HGBC_model_2024-05-29_12-08-48


## Model Quick Assessment
Get performance metrics for a given model.
Set the target variable by changing 'target' to LF22_FVT/LF22_FVC/LF22_FVH/LF22_F40 and run the celll. 

In [63]:
seed = 1234

# Define the target model to assess
target = 'LF22_FVT'

# Dictionary to define predictors based on target
predictors_dict = {
    'LF22_FVT' : ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH'],
    'LF22_FVC' : ['LF22_FVT', 'LF20_FVC', 'LF22_FDST', 'ZONE', 'BPS_NAME'],
    'LF22_FVH' : ['LF22_FVT', 'LF20_FVH', 'LF22_FDST', 'ZONE', 'BPS_NAME'],
    'LF22_F40' : ['LF22_FVT', 'LF22_FVH', 'LF22_FVC', 'LF22_FDST', 'ZONE', 'BPS_FRG_NE']
}

# Define train/test split proportion
train_frac = 0.7
test_frac = 0.3

# Define path to sample points
sample_points_fpath =  os.path.join(paths_dict['sample_points_dir'], sample_points_fnames[target])

# Read in and prepare data
sample_points = data_prep(sample_points_fpath, target)

# Perform Train/Test Split
train, test = train_test_split(sample_points, train_size=train_frac, test_size=test_frac, 
                               random_state=seed, shuffle=True, stratify=sample_points[target])

# Train model
model = fit_model(train_data=train, predictors=predictors_dict[target], target=target)

# Evaluate Model
print(target)
print("Score on the Training Set: ")
results = eval_model(model=model, test_data=train, predictors=predictors_dict[target], target=target)

print("Score on the test set: ")
results = eval_model(model=model, test_data=test, predictors=predictors_dict[target], target=target)

LF22_FVT
Score on the Training Set: 
Predictors: ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']
Accuracy: 0.996
Balanced Accuracy: 0.998
Recall: 0.998
Precision: 0.989
F1: 0.993
Score on the test set: 
Predictors: ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']
Accuracy: 0.996
Balanced Accuracy: 0.998
Recall: 0.998
Precision: 0.992
F1: 0.995


In [ ]:
# Save out results
model_type = 'HGBC'
## Concatenate predictions to test set
test[f'PRED_{target}'] = results['predictions']

## Create dataframe of metrics
metrics_df = pd.DataFrame(columns=['accuracy', 'balanced accuracy', 'recall', 'precision' ,'f1 score', 'attributes', 'model_type'])
metrics_df.loc[0] = results['metrics'] + [model_type]

## Save out the dataframes
out_dir = make_dir(paths_dict['out_base_dir'], new_dir_name=f'{target}_model_results')
preds_out_fname = f"Predictions_{target}_{model_type}.csv"
test.to_csv(os.path.join(out_dir, preds_out_fname))

metrics_out_fname = f"Metrics_{target}_{model_type}.csv"
metrics_df.to_csv(os.path.join(out_dir, metrics_out_fname))

# Save out model to results dir
datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_fname = f"{target}_{model_type}_model_{datetime}"
dump(model, os.path.join(out_dir, model_fname))